In [1]:
import os

import tifffile as tiff
import starfish
from starfish.types import Axes

In [8]:
def download_data(
    outdir: str,
    fov_idx: str = 'fov_001'
):
    # create output directory:
    try:
        os.mkdir(outdir)
    except:
        pass
    # Load experiment from bucket:
    exp = starfish.Experiment.from_json(
        "https://d2nhj9g34unfro.cloudfront.net/browse/formatted/20180926/iss_breast/experiment.json")
    # Select fov of interest:
    fov = exp[fov_idx]
    # Iterate over four rounds and four channels:
    image_stack = fov.get_image("primary")
    for r in range(4):
        for ch in range(4):
            img = image_stack.sel({Axes.ROUND: r, Axes.CH: ch}).xarray.squeeze()
            tiff.imwrite(os.path.join(outdir, f"r{r}_ch{ch}.tif"), img)
    
    # Write nuclei and anchor dots:
    nuclei_stack = fov.get_image("nuclei").xarray.squeeze()
    tiff.imwrite(os.path.join(outdir, "nuclei.tif"), nuclei_stack)
    
    dots_stack = fov.get_image("dots").xarray.squeeze()
    tiff.imwrite(os.path.join(outdir, "dots.tif"), dots_stack)
    
    # Save codebook:
    exp.codebook.to_json(os.path.join(outdir, 'codebook_minimaldataset.json'))

In [9]:
download_data('/scratch/segonzal/MinimalData')

100%|██████████| 1/1 [00:00<00:00, 60.99it/s]
